# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create Weather DataFrame

weather_df = pd.read_csv('weather_data_output.csv')
#weather_df = pd.read_csv('../output_data/cities.csv')
weather_df.dropna()
del weather_df ["Unnamed: 0"]
weather_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Hong Kong,HK,2020-03-15 17:06:59,22.29,114.16,20,72,68.00,10.29
1,Havøysund,NO,2020-03-15 17:07:36,71.00,24.66,57,67,21.20,5.82
2,Arrondissement de Toul,FR,2020-03-15 17:07:41,48.67,5.92,97,44,59.00,12.75
3,Atuona,PF,2020-03-15 17:08:04,-9.80,-139.03,17,67,81.84,9.04
4,Chui,UY,2020-03-15 17:08:06,-33.70,-53.46,100,56,73.20,10.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import os
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations and Fill NaN values and convert to float
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store Humidity values and Fill NaN values and convert to float
humidity = weather_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create Filtered Weather DataFrame 
ideal_weather_df = pd.DataFrame(weather_df)

# Fit Ideal Weather Conditions based on Cloudiness, Max Temp and Wind Speed
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Cloudiness (%)"] == 0)]                                               
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp (F)"] < 80) & (ideal_weather_df["Max Temp (F)"] > 70)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Wind Speed (mph)"] < 10)]    
                                              
# Drop any rows with null values and reset index
ideal_weather_df.dropna()
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
ideal_weather_df                              

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Nhulunbuy,AU,2020-03-15 17:09:21,-12.23,136.77,0,88,77.00,7.05
1,Necochea,AR,2020-03-15 17:11:01,-38.55,-58.74,0,70,72.00,8.01
2,Kununurra,AU,2020-03-15 17:06:26,-15.77,128.73,0,83,73.40,5.66
3,Katherine,AU,2020-03-15 17:11:10,-14.47,132.27,0,78,75.20,4.90
4,Viedma,AR,2020-03-15 17:19:24,-40.81,-63.00,0,18,76.98,2.46
5,Mount Isa,AU,2020-03-15 17:22:09,-20.73,139.50,0,31,77.00,8.05
6,Jalu,LY,2020-03-15 17:23:26,29.03,21.55,0,17,73.20,8.23
7,Catuday,PH,2020-03-15 17:23:58,16.29,119.81,0,75,79.68,5.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create New Dataframe 
hotel_df = ideal_weather_df

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Nhulunbuy,AU,2020-03-15 17:09:21,-12.23,136.77,0,88,77.00,7.05,
1,Necochea,AR,2020-03-15 17:11:01,-38.55,-58.74,0,70,72.00,8.01,
2,Kununurra,AU,2020-03-15 17:06:26,-15.77,128.73,0,83,73.40,5.66,
3,Katherine,AU,2020-03-15 17:11:10,-14.47,132.27,0,78,75.20,4.90,
4,Viedma,AR,2020-03-15 17:19:24,-40.81,-63.00,0,18,76.98,2.46,
5,Mount Isa,AU,2020-03-15 17:22:09,-20.73,139.50,0,31,77.00,8.05,
6,Jalu,LY,2020-03-15 17:23:26,29.03,21.55,0,17,73.20,8.23,
7,Catuday,PH,2020-03-15 17:23:58,16.29,119.81,0,75,79.68,5.53,


In [7]:
hotel_names = []


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # geocoordinates
    target_coordinates = f"{latitude},{longitude}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "Hotel"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}    
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    vacation_response = requests.get(base_url, params=params)

    # convert response to json
    places_data = vacation_response.json()

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    # extract results
    results = places_data['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: Nhulunbuy.
Closest hotel is Walkabout Lodge & Tavern.
Retrieving Results for Index 1: Necochea.
Closest hotel is Hotel Ñikén Spa & Business Center.
Retrieving Results for Index 2: Kununurra.
Closest hotel is Hotel Kununurra.
Retrieving Results for Index 3: Katherine.
Closest hotel is ibis Styles Katherine.
Retrieving Results for Index 4: Viedma.
Closest hotel is Alta Patagonia Apart.
Retrieving Results for Index 5: Mount Isa.
Closest hotel is Isa Hotel.
Retrieving Results for Index 6: Jalu.
Closest hotel is Jalu Hotel.
Retrieving Results for Index 7: Catuday.
Closest hotel is Hotel G Beach Resort and Restobar.


In [8]:
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Nhulunbuy,AU,2020-03-15 17:09:21,-12.23,136.77,0,88,77.00,7.05,Walkabout Lodge & Tavern
1,Necochea,AR,2020-03-15 17:11:01,-38.55,-58.74,0,70,72.00,8.01,Hotel Ñikén Spa & Business Center
2,Kununurra,AU,2020-03-15 17:06:26,-15.77,128.73,0,83,73.40,5.66,Hotel Kununurra
3,Katherine,AU,2020-03-15 17:11:10,-14.47,132.27,0,78,75.20,4.90,ibis Styles Katherine
4,Viedma,AR,2020-03-15 17:19:24,-40.81,-63.00,0,18,76.98,2.46,Alta Patagonia Apart
5,Mount Isa,AU,2020-03-15 17:22:09,-20.73,139.50,0,31,77.00,8.05,Isa Hotel
6,Jalu,LY,2020-03-15 17:23:26,29.03,21.55,0,17,73.20,8.23,Jalu Hotel
7,Catuday,PH,2020-03-15 17:23:58,16.29,119.81,0,75,79.68,5.53,Hotel G Beach Resort and Restobar


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))